In [1]:
import pandas as pd
datafile='d:/pydata/data.xls'
typelabel ={u'肝气郁结证型系数':'A', u'热毒蕴结证型系数':'B', u'冲任失调证型系数':'C', u'气血两虚证型系数':'D', u'脾胃虚弱证型系数':'E', u'肝肾阴虚证型系数':'F'}
k=4
file=pd.read_excel(datafile)
key=list(typelabel.keys())
result=pd.DataFrame()
from __future__ import print_function
from sklearn.cluster import KMeans
if __name__ =='__main__':
    for i in range(len(key)):
            print('ongoing "%s "KMean cluster' % key[i])
            kmodel=KMeans(n_clusters=k,n_jobs=4)
            kmodel.fit(file[[key[i]]].as_matrix())
            r1=pd.DataFrame(kmodel.cluster_centers_,columns=[typelabel[key[i]]])
            r2=pd.Series(kmodel.labels_).value_counts()
            r2=pd.DataFrame(r2,columns=[typelabel[key[i]]+'n'])
            r=pd.concat([r1,r2],axis=1).sort(typelabel[key[i]])
            r.index=[1,2,3,4]
            r[typelabel[key[i]]]=pd.rolling_mean(r[typelabel[key[i]]],2)
            r[typelabel[key[i]]][1]=0.0
            result=result.append(r.T)
            result=result.sort()
print(result)
            
 

    
    
    


ongoing "肝气郁结证型系数 "KMean cluster


D:\pythonanaconda\lib\site-packages\ipykernel\__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
D:\pythonanaconda\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
D:\pythonanaconda\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\pythonanaconda\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: sort(....) is deprecated, use sort_index(.....)


ongoing "热毒蕴结证型系数 "KMean cluster
ongoing "冲任失调证型系数 "KMean cluster
ongoing "气血两虚证型系数 "KMean cluster
ongoing "脾胃虚弱证型系数 "KMean cluster
ongoing "肝肾阴虚证型系数 "KMean cluster
        1           2           3           4
A     0.0    0.178698    0.257724    0.351843
An  240.0  356.000000  281.000000   53.000000
B     0.0    0.153543    0.298217    0.489954
Bn  342.0  380.000000  179.000000   29.000000
C     0.0    0.202149    0.289061    0.423537
Cn  297.0  394.000000  204.000000   35.000000
D     0.0    0.172281    0.251683    0.359353
Dn  283.0  375.000000  228.000000   44.000000
E     0.0    0.152698    0.257873    0.376062
En  273.0  319.000000  245.000000   93.000000
F     0.0    0.179143    0.261386    0.354643
Fn  200.0  237.000000  265.000000  228.000000


In [6]:
#-*- coding: utf-8 -*-
from __future__ import print_function
import pandas as pd

#自定义连接函数，用于实现L_{k-1}到C_k的连接
def connect_string(x, ms):
  x = list(map(lambda i:sorted(i.split(ms)), x))
  l = len(x[0])
  r = []
  for i in range(len(x)):
    for j in range(i,len(x)):
      if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
        r.append(x[i][:l-1]+sorted([x[j][l-1],x[i][l-1]]))
  return r

#寻找关联规则的函数
def find_rule(d, support, confidence, ms = u'--'):
  result = pd.DataFrame(index=['support', 'confidence']) #定义输出结果
  
  support_series = 1.0*d.sum()/len(d) #支持度序列
  column = list(support_series[support_series > support].index) #初步根据支持度筛选
  k = 0
  
  while len(column) > 1:
    k = k+1
    print(u'\n正在进行第%s次搜索...' %k)
    column = connect_string(column, ms)
    print(u'数目：%s...' %len(column))
    sf = lambda i: d[i].prod(axis=1, numeric_only = True) #新一批支持度的计算函数
    
    #创建连接数据，这一步耗时、耗内存最严重。当数据集较大时，可以考虑并行运算优化。
    d_2 = pd.DataFrame(list(map(sf,column)), index = [ms.join(i) for i in column]).T
    
    support_series_2 = 1.0*d_2[[ms.join(i) for i in column]].sum()/len(d) #计算连接后的支持度
    column = list(support_series_2[support_series_2 > support].index) #新一轮支持度筛选
    support_series = support_series.append(support_series_2)
    column2 = []
    
    for i in column: #遍历可能的推理，如{A,B,C}究竟是A+B-->C还是B+C-->A还是C+A-->B？
      i = i.split(ms)
      for j in range(len(i)):
        column2.append(i[:j]+i[j+1:]+i[j:j+1])
    
    cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列
 
    for i in column2: #计算置信度序列
      cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))]/support_series[ms.join(i[:len(i)-1])]
    
    for i in cofidence_series[cofidence_series > confidence].index: #置信度筛选
      result[i] = 0.0
      result[i]['confidence'] = cofidence_series[i]
      result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]
  
  result = result.T.sort_values(['confidence','support'], ascending = False) #结果整理，输出
  print(u'\n结果为：')
  print(result)
  
  return result

In [7]:
import time 
from __future__ import print_function
datafile2='d:/pydata/apriori.txt'
data=pd.read_csv(datafile2,header = None, dtype=object)
start=time.clock()
print("transfrom original dataframe into 0-1 matrix")
ct = lambda x : pd.Series(1, index = x[pd.notnull(x)])
b = map(ct, data.as_matrix()) 
c=list(b)
data=pd.DataFrame(c).fillna(0)
end = time.clock() 
print('\n complete transforming use ：%0.2f seconds' %(end-start))
del b
support=0.06
confidence=0.75
ms='---'
start=time.clock()
print("begin to searching aprior rule...")
find_rule(data,support,confidence,ms)
end=time.clock()
print('\n complete searching use ：%0.2f seconds' %(end-start))


transfrom original dataframe into 0-1 matrix

 complete transforming use ：0.40 seconds
begin to searching aprior rule...

正在进行第1次搜索...
数目：276...

正在进行第2次搜索...
数目：947...

正在进行第3次搜索...
数目：41...

结果为：
                    support  confidence
A3---F4---H4       0.078495    0.879518
C3---F4---H4       0.075269    0.875000
B2---F4---H4       0.062366    0.794521
C2---E3---D2       0.092473    0.754386
D2---F3---H4---A2  0.062366    0.753247

 complete searching use ：1.51 seconds
